
# Análise de Sentimento de Tweets em Português 
**Resumo:** Este notebook mostra, passo a passo, como carregar um dataset de tweets em português, pré‑processar, tunar um modelo Transformer (BERTimbau) com Keras/TensorFlow para classificação de sentimento. 




### Instalação de dependências para Google colab

In [ ]:

# Execute apenas se precisar instalar dependências
# Em muitos ambientes já configurados, estas bibliotecas já estarão presentes.
# !pip install -q transformers datasets 'tensorflow>=2.10' scikit-learn matplotlib --upgrade


### Imports e configurações iniciais

Esta célula importa as bibliotecas e define seed


In [ ]:

import os
import random
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TFAutoModel
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)



### Carregar o dataset `eduagarcia/tweetsentbr_fewshot`

Nesta célula carregamos o dataset do Hugging Face Hub. Nesse dataset, só tem 25 twwets de treino. Por isso vamos pegar alguns tweets de teste e mandar para o treino


In [ ]:

# Carrega o dataset do Hugging Face Hub
ds = load_dataset('eduagarcia/tweetsentbr_fewshot')
train_ds = ds["train"]
test_ds = ds["test"]

# Número de exemplos de teste a mover
n_move = len(test_ds) // 2  # metade

# Dividimos o test dataset em duas partes
test_split = test_ds.train_test_split(test_size=n_move, seed=SEED)

# A primeira parte (metade) vai para o treino
new_train_ds = Dataset.from_dict({
    k: list(train_ds[k]) + list(test_split["train"][k]) for k in train_ds.features
})

# A segunda metade continua como teste
new_test_ds = test_split["test"]

# Criamos um novo DatasetDict
new_dataset = DatasetDict({
    "train": new_train_ds,
    "test": new_test_ds
})

ds = new_dataset

print('Keys do DatasetDict (splits):', ds.keys())
split_name = list(ds.keys())[0]
print('Colunas disponíveis no split "{}":'.format(split_name), ds[split_name].column_names)

# Mostra os 3 primeiros exemplos para entendimento do formato
print('Exemplos de entrada:')
for i in range(3):
    print(i+1, ds[split_name][i])



### Atribuir os nomes das colunas e mapear os rótulos em números


In [ ]:
TEXTO = 'sentence'
LABEL = 'label'

ROTULOS = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

train_df = ds['train'].to_pandas()
test_df = ds['test'].to_pandas()

train_df[LABEL] = train_df[LABEL].map(ROTULOS)
test_df[LABEL] = test_df[LABEL].map(ROTULOS)

# converter labels para numpy int32
y_train = train_df[LABEL].to_numpy(dtype='int32')
y_test = test_df[LABEL].to_numpy(dtype='int32')



### Só mostra algumas entradas


In [ ]:
print('Amostra usada para treino:', len(train_df), 'teste:', len(test_df))
train_df.head(10)



### Tokenização com o tokenizer BERTimbau

Nesta célula tokenizamos os textos usando `AutoTokenizer`. Configurações importantes:

- `max_length=128` (tweets são curtos)
- `padding='max_length'` para facilitar a criação de batches estáveis
- `truncation=True` para cortar textos muito longos


In [ ]:

model_name = 'neuralmind/bert-base-portuguese-cased'  # BERTimbau
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_df(df, max_length=128):
    enc = tokenizer(list(df[TEXTO]), padding='max_length', truncation=True, max_length=max_length, return_tensors='np')
    return enc

train_enc = tokenize_df(train_df)
test_enc = tokenize_df(test_df)

print('Exemplo shapes: input_ids', train_enc['input_ids'].shape, 'attention_mask', train_enc['attention_mask'].shape)
print('Labels shape:', y_train.shape)


### Criar `tf.data.Dataset` a partir dos encodings


O modelo espera tensores numéricos (`input_ids`, `attention_mask`) e labels inteiros. Aqui criamos `tf.data.Dataset` com tipo correto.


In [ ]:

# Função auxiliar para criar dataset TF
def make_tf_dataset(encodings, labels, batch_size=16, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
        },
        labels
    ))
    if shuffle:
        dataset = dataset.shuffle(1000, seed=SEED)
    return dataset.batch(batch_size)

batch_size = 16
train_ds = make_tf_dataset(train_enc, y_train, batch_size=batch_size, shuffle=True)
test_ds = make_tf_dataset(test_enc, y_test, batch_size=batch_size, shuffle=False)

# Sanity check: shapes from one batch
for x_batch, y_batch in train_ds.take(1):
    print('Batch input_ids shape:', x_batch['input_ids'].shape)
    print('Batch attention_mask shape:', x_batch['attention_mask'].shape)
    print('Batch labels shape:', y_batch.shape)


### Criar e compilar o modelo de classificação (TFAutoModelForSequenceClassification)

Aqui carregamos a versão TensorFlow do modelo com uma cabeça de classificação (`num_labels` depende do dataset).Usamos um `learning_rate` pequeno porque estamos fine‑tuning de um modelo pré‑treinado.


In [ ]:

# número de classes detectado a partir do mapeamento
num_labels = len(sorted(set(y_train.tolist() + y_test.tolist())))
print('Número de classes:', num_labels)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.summary()


### Treinamento (fine‑tuning)


**Atenção:** em CPU o treinamento pode ser lento. Mantemos `epochs=1` por padrão só para fins de demonstração em sala. Se você tiver GPU disponível, aumente `epochs` maiores. Mas experimente antes porque demora


In [ ]:

epochs = 1
history = model.fit(train_ds, validation_data=test_ds, epochs=epochs)


### Avaliação e predições de exemplo

Geramos métricas e mostramos algumas predições manuais para sentir o comportamento do modelo. Até fazer predição demora se você não tiver uma GPU disponível


In [ ]:

# Avaliar no conjunto de teste
results = model.evaluate(test_ds)
print('Teste (loss, accuracy):', results)

# Função de predição simples para textos novos
def predict_text(texts):
    enc = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors='tf')
    logits = model(**enc).logits
    probs = tf.nn.softmax(logits, axis=-1).numpy()
    preds = np.argmax(probs, axis=1)
    return preds, probs

# Exemplos rápidos
examples = [
    'Amei o atendimento, voltarei sempre!',
    'Péssima experiência, demorou demais e o produto veio danificado.',
    'O app funciona, mas poderia melhorar.'
]

preds, probs = predict_text(examples)
for t, p, pr in zip(examples, preds, probs):
    print('\nTexto:', t)
    print('Predito (id):', int(p), ' — probs:', np.round(pr,3))